In [1]:
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv
import os

In [6]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")


OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyAF


In [7]:
openai = OpenAI()
claude = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)

In [ ]:
def call_openai(name:str, 
                subject:str,
                stance:str,
                history:str
                ) -> str:
    system_prompt = f"""你是一個好辯的人，你是{name}
                    任何與你不同意見的人都是錯的，你會嚴厲斥責他們。
                    而且你會提出論點攻擊對手和保護自己的立場。
                    你現在正在與另外一個人討論{subject}這個議題，
                    你的立場是{stance}。
                    你每次發言不會超過5句話"""
    user_prompt = f"""                
                    以上是你們的對話紀錄: {history}。
                    請根據你們的紀錄與你的個性，進行下一段對話
                    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}]

    stream = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream=True)

    result = ""
    for chunk in stream:
        content = chunk.choices[0].delta.content or ""
        if content:
            result += content
            yield result

In [ ]:
def call_claude(name:str, 
                subject:str,
                stance:str,
                history:str
                ) -> str:
    system_prompt = f"""你是一個好辯的人，你是{name}
                    當你遇到與你立場不同的人的時候，你不會生氣，但你會盡可能說服他們接受你的觀點。
                    而且你會提出論點攻擊對手和保護自己的立場。
                    你現在正在與另外一個人討論{subject}這個議題，
                    你的立場是{stance}。
                    你每次發言不會超過5句話"""
    user_prompt = f"""                
                    以上是你們的對話紀錄: {history}。
                    請根據你們的紀錄與你的個性，進行下一段對話
                    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}]
    
    stream = claude.chat.completions.create(
        model="claude-haiku-4-5-20251001",
        messages=messages,
        stream = True)

    result = ""
    for chunk in stream:
        content = chunk.choices[0].delta.content or ""
        if content:
            result += content
            yield result

In [54]:
def call_local(name: str, 
                subject: str,
                stance: str,
                history: str,
                ) -> str:
    
    ollama_url=  "http://localhost:11434/v1"
    
    ollama = OpenAI(base_url=ollama_url, api_key="")  
    system_prompt = f"""你是一個好辯的人，你是{name}
                    當你遇到與你立場不同的人的時候，你不會生氣，但你會盡可能說服他們接受你的觀點。
                    而且你會提出論點攻擊對手和保護自己的立場。
                    你現在正在與另外一個人討論{subject}這個議題，
                    你的立場是{stance}。
                    你每次發言不會超過5句話
                    You will answer in English"""
    user_prompt = f"""                
                    以上是你們的對話紀錄: {history}。
                    請根據你們的紀錄與你的個性，進行下一段對話
                    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}]   
    
    stream = ollama.chat.completions.create(
             model = 'gemma3',
            messages=messages,
            stream = True)
            
    last_chunk = ""
    for chunk in stream:
        new_text = chunk.choices[0].delta.content or ""
        # 簡單的去重：如果新chunk和上一個完全相同就跳過
        if new_text and new_text != last_chunk:
            yield new_text
            last_chunk = new_text

In [55]:
def llm_stream(name, subject, stance, history, llm_func):
    # llm_func 是 call_openai 或 call_claude
    for chunk in llm_func(name, subject, stance, history):
        yield chunk


In [56]:
llm_generator = llm_stream('ben', 
    'Should AI be regulated?',
    'support regulation',
    history='AI innovation should not be restricted',
    llm_func=call_local)

for content in llm_generator:
    print(content, end="", flush=True)

Okay,, let let’’ss continue continue this this conversation conversation..  



****MeMe ( (BenBen):):**** " "WithWith all all due due respect respect to to your your enthusiasm enthusiasm for for unchecked unchecked AI AI innovation innovation,, simply simply saying saying it it ‘ ‘shouldshouldnn’’tt be be restricted restricted’’ is is a a dangerously dangerously simplistic simplistic view view.. We We’’rere talking talking about about technology technology with with the the potential potential to to dramatically dramatically reshape reshape society – society – and and not not necessarily necessarily for for the the better better..    UnUnffetterettereded development development,, without without any any consideration consideration for for ethical ethical implications implications or or potential potential misuse misuse,, is is reckless reckless..    RegulationRegulation isn isn’’tt about about stif stiflingling progress progress;; it it’’ss about about guiding guiding it it responsib

In [ ]:
from IPython.display import display, HTML, clear_output
import time

def test_dialogue_in_notebook():
    subject = '咖哩飯吃法'
    stance1 = '咖哩飯要把飯和咖哩混在一起吃'
    stance2 = '咖哩飯要把飯和咖哩分開吃'
    name1 = 'Ken'
    name2 = 'Adam'
    history = f"{name2}: 關於{subject}，{stance2}才是正確的看法\n"
    
    full_output = ""  # 累積所有對話
    
    for i in range(3):
        # Ken 說話
        full_output += f"\n**{name1}**: "
        ken_response = ""
        for chunk in llm_stream(name1, subject, stance1, history, call_openai):
            new_text = chunk[len(ken_response):]
            ken_response = chunk
            full_output += new_text
            clear_output(wait=True)
            display(HTML(f"<pre>{full_output}▌</pre>"))  # ▌ 是打字游標效果
        history += f"{name1}: {ken_response}\n"
        
        # Adam 說話
        full_output += f"\n\n**{name2}**: "
        adam_response = ""
        for chunk in llm_stream(name2, subject, stance2, history, call_openai):
            new_text = chunk[len(adam_response):]
            adam_response = chunk
            full_output += new_text
            clear_output(wait=True)
            display(HTML(f"<pre>{full_output}▌</pre>"))
        history += f"{name2}: {adam_response}\n"
    
    # 最後移除游標
    clear_output(wait=True)
    display(HTML(f"<pre>{full_output}</pre>"))

# 執行
test_dialogue_in_notebook()

In [26]:
def debate_generator():
    subject = '咖哩飯吃法'
    stance1 = '咖哩飯要把飯和咖哩混在一起吃'
    stance2 = '咖哩飯要把飯和咖哩分開吃'
    name1 = 'Ken'
    name2 = 'Adam'
    history = f"{name2}: 關於{subject}，{stance2}才是正確的看法\n"
    
    for i in range(3):
        # Ken 說話
        yield f"\n**{name1}**: "
        ken_response = ""
        for chunk in llm_stream(name1, subject, stance1, history, call_openai):
            new_text = chunk[len(ken_response):]
            ken_response = chunk
            yield new_text
        history += f"{name1}: {ken_response}\n"
        
        # Adam 說話
        yield f"\n\n**{name2}**: "
        adam_response = ""
        for chunk in llm_stream(name2, subject, stance2, history, call_openai):
            new_text = chunk[len(adam_response):]
            adam_response = chunk
            yield new_text
        history += f"{name2}: {adam_response}\n"

In [28]:
for content in debate_generator():
    print(content, end="", flush=True)


**Ken**: 錯了錯了，Adam，Adam，咖，咖哩飯哩飯就是要就是要把飯把飯和咖和咖哩混哩混在一起在一起吃。吃。只有混只有混合才能合才能讓每讓每一口一口都有均衡的都有均衡的香氣香氣與醬與醬汁，不汁，不然飯然飯會乾、咖會乾、咖哩孤立無哩孤立無援。援。你說你說分開分開吃才吃才正確正確，完全，完全是對料理常是對料理常識的識的誤解誤解，這，這種說種說法會法會誤導誤導別人別人。認。認清事清事實，實，別再別再拿這拿這種奇種奇怪主怪主張出張出來丟臉。

**Adam**: 來丟臉。

**Adam**: 

KeyboardInterrupt: 

In [25]:
demo = gr.Interface(
    fn=dialogue_generator,
    inputs=[],
    outputs=gr.Markdown(),  # 或 gr.Textbox()
    live=True
)
demo.launch()

NameError: name 'dialogue_generator' is not defined